<p style="font-weight:bold;"> <span style="font-size: 36px"> Ifrs Variable </span> </p>

In [0]:
#!eval-notebook "Data/InitSystemorphToMemoryForTesting"

# BenchMarks

In [0]:
var pathToBm = "./Data/IfrsVariableBenchmarks/";

In [0]:
public record BenchmarkMetadata(string FileName, string ReportingNode, int Year, int Month, string Scenario = null){} 

## BenchMark to test Value0

In [0]:
var bmFiles = new BenchmarkMetadata[]{
    //2020 Q4
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__A.csv"   , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__AA.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__APA.csv" , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__BE.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__BEPA.csv", "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__C.csv"   , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__DA.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__F.csv"   , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__OA.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__RA.csv"  , "CH", 2020, 12),
    // 2021 Q1
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__A.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__AA.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__APA.csv"  , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__BE.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__BEPA.csv" , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__C.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__DA.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__F.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__L.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__LR.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__OA.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__RA.csv"   , "CH", 2021, 3),
    // 2020 Q4
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_BE.csv"   , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_BEPA.csv" , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_A.csv"    , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_APA.csv"  , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_DA.csv"   , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_C.csv"    , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_F.csv"    , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_RA.csv"   , "CH", 2020, 12, "MTUP10pct"),
};

## BenchMark to test Projections

In [0]:
var bmFilesTestProjection = new BenchmarkMetadata[]{
    // 2021 Q1
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__BE_Projection_DTP.csv"   , "CH", 2021, 3)
};

In [0]:
var excludedDataNodes = new string[]{};
var projectionDataNodes = new string[]{"DTP1.1"};

# Functions

In [0]:
public bool isCorrectValue(double value, bool isProjectedValue) => isProjectedValue? Math.Abs(value) <= BenchmarkPrecision : true;

In [0]:
var listIfrsValueProperties = new string[]{nameof(IfrsVariable.Value), nameof(IfrsVariable.Value1), nameof(IfrsVariable.Value2), nameof(IfrsVariable.Value3), nameof(IfrsVariable.Value4), nameof(IfrsVariable.Value5),
nameof(IfrsVariable.Value6), nameof(IfrsVariable.Value7), nameof(IfrsVariable.Value8), nameof(IfrsVariable.Value9), nameof(IfrsVariable.Value10), nameof(IfrsVariable.Value11),
nameof(IfrsVariable.Value12), nameof(IfrsVariable.Value13), nameof(IfrsVariable.Value14), nameof(IfrsVariable.Value15), nameof(IfrsVariable.Value16), nameof(IfrsVariable.Value17),
nameof(IfrsVariable.Value18)};

In [0]:
public static NumberStyles AllowedNumberStyles = NumberStyles.Float | NumberStyles.AllowLeadingWhite | NumberStyles.AllowTrailingWhite | NumberStyles.AllowLeadingSign | NumberStyles.AllowTrailingSign | NumberStyles.AllowDecimalPoint;

In [0]:
public double ParseIfrsVariableValue(IDataRow row, string nameOfIfrsVariableValue)
{
    return (row.Table.Columns.Contains(nameOfIfrsVariableValue)
                        ? (double.TryParse(row[nameOfIfrsVariableValue]?.ToString(), AllowedNumberStyles, CultureInfo.InvariantCulture, out var value) 
                                ? value 
                                : throw new Exception(nameOfIfrsVariableValue + " cannot be parsed."))
                        : default(double));
}

In [0]:
public IfrsVariable CreateVariable(IDataRow row, Guid partition)
    => new IfrsVariable(){AocType = row.Field<string>(nameof(IfrsVariable.AocType)),
                          Partition = partition,
                          Novelty = row.Field<string>(nameof(IfrsVariable.Novelty)),
                          DataNode = row.Field<string>(nameof(IfrsVariable.DataNode)),
                          EstimateType = row.Field<string>(nameof(IfrsVariable.EstimateType)),
                          AmountType = row.Field<string>(nameof(IfrsVariable.AmountType)),
                          AccidentYear = Int32.TryParse((row.Field<string>(nameof(IfrsVariable.AccidentYear))), out var accidentYear) ? accidentYear : (int?)null,
                          EconomicBasis = row.Field<string>(nameof(IfrsVariable.EconomicBasis)),
                          Value = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value)), 
                          Value1 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value1)), 
                          Value2 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value2)),
                          Value3 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value3)), 
                          Value4 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value4)), 
                          Value5 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value5)),                          
                          Value6 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value6)), 
                          Value7 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value7)), 
                          Value8 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value8)),
                          Value9 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value9)), 
                          Value10 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value10)), 
                          Value11 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value11)),                          
                          Value12 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value12)), 
                          Value13 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value13)), 
                          Value14 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value14)),
                          Value15 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value15)), 
                          Value16 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value16)), 
                          Value17 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value17)),
                          Value18 = (-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value18))};

In [0]:
public class BenchmarkTestResult
{
    public string ErrorMessage {get; set;}
    public double ExpectedValue {get; set;}
    public double ComputedValue {get; set;}
    public BenchmarkTestResult (string message)
    {
        ErrorMessage = message;
    }
    public BenchmarkTestResult (string message, double expValue, double compValue)
    {
        ErrorMessage = message;
        ExpectedValue = expValue;
        ComputedValue = compValue;    
    }
}

In [0]:
public List<BenchmarkTestResult> CompareAgainstBm(List<BenchmarkTestResult> errors, IEnumerable<IfrsVariable> expected, IEnumerable<IfrsVariable> computed)
{
    var comparer = EqualityComparer<IfrsVariable>.Instance;
    var expectedNotComputed = expected.Where(x => Math.Abs(x.Value) > BenchmarkPrecision).Except(computed, comparer);
    var computedNotExpected = computed.Where(x => Math.Abs(x.Value) > BenchmarkPrecision).Except(expected, comparer);
    if (expectedNotComputed.Any())
        foreach (var variable in expectedNotComputed)
            errors.Add(new BenchmarkTestResult("Extra expected variable for: Partition " + variable.Partition + ", " + variable.ToIdentityString()));
    if (computedNotExpected.Any())
        foreach (var variable in computedNotExpected)
            errors.Add(new BenchmarkTestResult("Extra computed variable for: Partition " + variable.Partition + ", " + variable.ToIdentityString()));

    var includingProperties = typeof(IfrsVariable).GetIdentityProperties().Select(x=>x.Name).ToArray();

    var benchmarkErrors = expected.Concat(computed)
            .GroupBy(x => new{Partition = x.Partition, IdentityString = x.ToStringWith(properties: includingProperties)})
            .Where(g => g.Count() > 1)
            .Select(g => new{ IfrsVariableKey = g.Key,
                              IsCorrect = new bool[]{isCorrectValue(g.Sum(i => i.Value), true), isCorrectValue(g.Sum(i => i.Value1), ProjectionCount > 1), isCorrectValue(g.Sum(i => i.Value2), ProjectionCount > 2), isCorrectValue(g.Sum(i => i.Value3),ProjectionCount > 3),
                                                        isCorrectValue(g.Sum(i => i.Value4), ProjectionCount > 4), isCorrectValue(g.Sum(i => i.Value5), ProjectionCount > 5), isCorrectValue(g.Sum(i => i.Value6), ProjectionCount > 6), isCorrectValue(g.Sum(i => i.Value7), ProjectionCount > 7), 
                                                        isCorrectValue(g.Sum(i => i.Value8), ProjectionCount > 8), isCorrectValue(g.Sum(i => i.Value9), ProjectionCount > 9), isCorrectValue(g.Sum(i => i.Value10), ProjectionCount > 10), isCorrectValue(g.Sum(i => i.Value11), ProjectionCount > 11), 
                                                        isCorrectValue(g.Sum(i => i.Value12), ProjectionCount > 12), isCorrectValue(g.Sum(i => i.Value13), ProjectionCount > 13), isCorrectValue(g.Sum(i => i.Value14), ProjectionCount > 14), isCorrectValue(g.Sum(i => i.Value15), ProjectionCount > 15),
                                                        isCorrectValue(g.Sum(i => i.Value16), ProjectionCount > 16), isCorrectValue(g.Sum(i => i.Value17), ProjectionCount > 17), isCorrectValue(g.Sum(i => i.Value18), ProjectionCount > 18)}})
            .SelectMany(y => y.IsCorrect.Select((isCorrect, ind) => !isCorrect? new BenchmarkTestResult("Value" + (ind != 0 ? ind.ToString() : "") + " does not match for Variable: Partition " + y.IfrsVariableKey.Partition + "," + y.IfrsVariableKey.IdentityString) : default(BenchmarkTestResult)))
            .Where(x => x != default(BenchmarkTestResult));

    errors.AddRange(benchmarkErrors);

    return errors;
}

# Test Runner

In [0]:
public bool ValidateMainTab(IDataRow mainRow, string reportingNode, int year, int month)
{
    if (!int.TryParse(mainRow[nameof(PartitionByReportingNodeAndPeriod.Year)]?.ToString(), NumberStyles.AllowLeadingWhite | NumberStyles.AllowTrailingWhite , CultureInfo.InvariantCulture, out var mainYear))
            throw new Exception("Value Year cannot be parsed."); 
    if (!int.TryParse(mainRow[nameof(PartitionByReportingNodeAndPeriod.Month)]?.ToString(), NumberStyles.AllowLeadingWhite | NumberStyles.AllowTrailingWhite , CultureInfo.InvariantCulture, out var mainMonth))
            throw new Exception("Value Month cannot be parsed."); 
    var mainReportingNode = mainRow[nameof(PartitionByReportingNodeAndPeriod.ReportingNode)].ToString();
    
    return reportingNode == mainReportingNode && year == mainYear && month == mainMonth;
}

In [0]:
public async Task<List<BenchmarkTestResult>> CheckIfrsVariablesValuesAsync(BenchmarkMetadata[] bmFiles)
{
    var errorList = new List<BenchmarkTestResult>();
    foreach (var g in bmFiles.GroupBy(x => (ReportingNode: x.ReportingNode, Year: x.Year, Month: x.Month, Scenario : x.Scenario)))
    {
        var reportingNode = g.Key.ReportingNode;
        var year = g.Key.Year;
        var month = g.Key.Month;
        var scenario = g.Key.Scenario;
    
        await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new {ReportingNode = reportingNode, Year = year, Month = month, Scenario = scenario});
        var partition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().ToArrayAsync())
                            .SingleOrDefault(x => x.ReportingNode == reportingNode && x.Year == year && x.Month == month && x.Scenario == scenario)?.Id ?? 
                            throw new Exception("Partition not found.");
    
        var computedVariablesByEstimateType = (await DataSource.Query<IfrsVariable>()
                                                            .Where(v => !excludedDataNodes.Contains(v.DataNode))
                                                            .Where(v => ProjectionCount > 1
                                                                                        ? projectionDataNodes.Contains(v.DataNode)
                                                                                        : !projectionDataNodes.Contains(v.DataNode))
                                                            .ToArrayAsync())
                                                            .ToDictionaryGrouped(x => x.EstimateType, x => x.ToArray());

        foreach (var bmfile in g)
        {
            var extension = System.IO.Path.GetExtension(bmfile.FileName);
            var stream = await Project.FileStorage.ReadAsync(bmfile.FileName);
            //Read Bm csv
            var bmDataTable = (await DataSetReader.ReadFromStream(stream).WithContentType(extension).ExecuteAsync()).DataSet;

            var mainTab = bmDataTable.Tables[Main].Rows.First();
            if(!ValidateMainTab(mainTab, reportingNode, year, month))
                throw new Exception("Metadata and main tab of the Benchmark are different.");

            var bmTableName = bmDataTable.Tables.Single(x => x.TableName != Main).TableName;
            var bmVariables = bmDataTable.Tables[bmTableName].Rows.Select(x => CreateVariable(x, partition));

            //Get the computed Data
            var computedVariables = computedVariablesByEstimateType.TryGetValue(bmTableName, out var ret) ? ret : Enumerable.Empty<IfrsVariable>();

            if(!computedVariables.Any())
                errorList.Add(new BenchmarkTestResult("No variables are computed for EstimateType: " + bmTableName, 0, 0));
            //CompareAgainstBm
            if (bmVariables.Any() && computedVariables.Any()) //TODO we are adding duplicates here
                errorList = errorList.Concat(CompareAgainstBm(errorList, bmVariables, computedVariables)).Distinct().ToList();
        }
    }
    return errorList;
}

## Test Value

In [0]:
ProjectionCount = 1;
var errorList = await CheckIfrsVariablesValuesAsync(bmFiles);

In [0]:
errorList.Count().Should().Be(0);

## Projection Test

When we set the ProjectionCount to 1, query on the DataSource ignores the projection data node DTP1.1, so this test <b>will fail</b>. At the moment this test is expected to succeed, when ProjectionCount is chosen to be in the range between 2 and 5, where the upper bound corresponds to the Projection = 4, which induces the calculation for the year 2022, month 12.

In [0]:
ProjectionCount = 5;
await Import.FromFile("../Files/TransactionalData/NominalCashflows_CH_2020_12_Projection.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync();
await Import.FromFile("../Files/TransactionalData/NominalCashflows_CH_2021_3_Projection.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync();

In [0]:
var projectionErrorList = await CheckIfrsVariablesValuesAsync(bmFilesTestProjection);

In [0]:
projectionErrorList.Count().Should().Be(0);